# Lesson 2: Guidelines for Prompting
https://learn.deeplearning.ai/chatgpt-prompt-eng/lesson/2/guidelines

In this lesson. we'll practice two prompting principles and their related tactics in order to write effective prompts for LLM.

In [1]:
import os

import openai
from dotenv import find_dotenv, load_dotenv

if not load_dotenv(find_dotenv()):
    raise RuntimeError("Set environment variables first!")

if key := os.getenv("OPENAI_API_KEY"):
    openai.api_key = key
    print("API key loaded")
else:
    raise RuntimeError("No API key found")

API key loaded


In [2]:
MODEL_NAME = "gpt-3.5-turbo"

In [3]:
def get_completion(prompt: str, model: str = MODEL_NAME) -> str:
    """Helper function to make it easier to use prompts and look at the
    generated outpus"""
    messages = [{"role": "user", "content": prompt}]

    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0.0,
    )

    return response.choices[0].message["content"]

Prompting Principles

1. Write clear and specific instructions
2. Give the model time to "think"


## Principle 1: Write clear and specific instructions

### Tactics 1: Use delimiters to clearly indicate distinct parts of the input.

In [4]:
text = """
You should express what you want a model to do by \
providing instructions that are as clear and \
specific as you can possibly make them. \
This will guide the model towards the desired output, \
and reduce the chances of receiving irrelevant \
or incorrect responses. Don't confuse writing a \
clear prompt with writing a short prompt. \
In many cases, longer prompts provide more clarity \
and context for the model, which can lead to \
more detailed and relevant outputs.
"""

prompt = f"""
Summarize the text delimited by triple backticks \
into a single sentence.
```{text}```
"""

print(get_completion(prompt=prompt))

To guide a model towards the desired output and minimize irrelevant or incorrect responses, it is important to provide clear and specific instructions, even if it means writing longer prompts that offer more clarity and context.


### Tactic 2: Ask for a structured output.

In [5]:
prompt = """
Generate a list of three made-up book titles along \
with their authors and genres.
Provide them in JSON format with the following keys:
book_id, title, author, genre.
"""

print(get_completion(prompt=prompt))

{
  "books": [
    {
      "book_id": 1,
      "title": "The Enigma of Elysium",
      "author": "Aria Nightshade",
      "genre": "Fantasy"
    },
    {
      "book_id": 2,
      "title": "Whispers in the Shadows",
      "author": "Evelyn Blackwood",
      "genre": "Mystery"
    },
    {
      "book_id": 3,
      "title": "Beyond the Veil",
      "author": "Lucas Stormrider",
      "genre": "Science Fiction"
    }
  ]
}


### Tactic 3: Ask the model to check whether conditions are satisfied.

In [6]:
text_1 = """
Making a cup of tea is easy! First, you need to get some \
water boiling. While that's happening, \
grab a cup and put a tea bag in it. Once the water is \
hot enough, just pour it over the tea bag. \
Let it sit for a bit so the tea can steep. After a \
few minutes, take out the tea bag. If you \
like, you can add some sugar or milk to taste. \
And that's it! You've got yourself a delicious \
cup of tea to enjoy.
"""

prompt = f"""
You will be provided with text delimited by triple quotes.
If it contains a sequence of instructions, \
re-write those instructions in the following format:

Step 1 - ...
Step 2 - …
…
Step N - …

If the text does not contain a sequence of instructions, \
then simply write \"No steps provided.\"

\"\"\"{text_1}\"\"\"
"""

print("Completion for Text 1:")
print(get_completion(prompt=prompt))

Completion for Text 1:
Step 1 - Get some water boiling.
Step 2 - Grab a cup and put a tea bag in it.
Step 3 - Pour the hot water over the tea bag.
Step 4 - Let the tea steep for a few minutes.
Step 5 - Take out the tea bag.
Step 6 - Add sugar or milk to taste.
Step 7 - Enjoy your cup of tea.


In [7]:
text_2 = """
The sun is shining brightly today, and the birds are \
singing. It's a beautiful day to go for a \
walk in the park. The flowers are blooming, and the \
trees are swaying gently in the breeze. People \
are out and about, enjoying the lovely weather. \
Some are having picnics, while others are playing \
games or simply relaxing on the grass. It's a \
perfect day to spend time outdoors and appreciate the \
beauty of nature.
"""

prompt = f"""
You will be provided with text delimited by triple quotes.
If it contains a sequence of instructions, \
re-write those instructions in the following format:

Step 1 - ...
Step 2 - …
…
Step N - …

If the text does not contain a sequence of instructions, \
then simply write \"No steps provided.\"

\"\"\"{text_2}\"\"\"
"""

print("Completion for Text 2:")
print(get_completion(prompt=prompt))

Completion for Text 2:
No steps provided.


### Tactic 4: "Few-shot" prompting

In [8]:
prompt = """
Your task is to answer in a consistent style.

<child>: Teach me about patience.

<grandparent>: The river that carves the deepest \
valley flows from a modest spring; the \
grandest symphony originates from a single note; \
the most intricate tapestry begins with a solitary thread.

<child>: Teach me about resilience.
"""

print(get_completion(prompt=prompt))

<grandparent>: Resilience is like a mighty oak tree that withstands the strongest storms, bending but never breaking. It is the unwavering spirit that rises again and again, even in the face of adversity.


## Principle 2: Give the model time to “think”

### Tactic 1: Specify the steps required to complete a task

In [9]:
text = """
In a charming village, siblings Jack and Jill set out on \
a quest to fetch water from a hilltop \
well. As they climbed, singing joyfully, misfortune \
struck—Jack tripped on a stone and tumbled \
down the hill, with Jill following suit. \
Though slightly battered, the pair returned home to \
comforting embraces. Despite the mishap, \
their adventurous spirits remained undimmed, and they \
continued exploring with delight.
"""

# example 1
prompt_1 = f"""
Perform the following actions:
1 - Summarize the following text delimited by triple \
backticks with 1 sentence.
2 - Translate the summary into French.
3 - List each name in the French summary.
4 - Output a json object that contains the following \
keys: french_summary, num_names.

Separate your answers with line breaks.

Text:
```{text}```
"""

print("Completion for prompt 1:")
print(get_completion(prompt=prompt_1))

Completion for prompt 1:
1 - Jack and Jill, siblings, go on a quest to fetch water from a well on a hill, but they both fall down the hill and return home slightly injured but still adventurous.

2 - Jack et Jill, frère et sœur, partent à la recherche d'eau d'un puits situé au sommet d'une colline, mais ils tombent tous les deux et rentrent chez eux légèrement blessés mais toujours aventureux.

3 - Jack, Jill.

4 - {
  "french_summary": "Jack et Jill, frère et sœur, partent à la recherche d'eau d'un puits situé au sommet d'une colline, mais ils tombent tous les deux et rentrent chez eux légèrement blessés mais toujours aventureux.",
  "num_names": 2
}


Ask for output in a specified format

In [10]:
prompt_2 = f"""
Your task is to perform the following actions:
1 - Summarize the following text delimited by
  <> with 1 sentence.
2 - Translate the summary into French.
3 - List each name in the French summary.
4 - Output a json object that contains the
  following keys: french_summary, num_names.

Use the following format:
Text: <text to summarize>
Summary: <summary>
Translation: <summary translation>
Names: <list of names in Italian summary>
Output JSON: <json with summary and num_names>

Text: <{text}>
"""

print("Completion for prompt 2:")
print(get_completion(prompt=prompt_2))

Completion for prompt 2:
Summary: Jack and Jill go on a quest to fetch water from a hilltop well, but they both fall down the hill and return home slightly battered but still adventurous.
Translation: Jack et Jill partent à la recherche d'eau d'un puits au sommet d'une colline, mais ils tombent tous les deux et rentrent chez eux légèrement blessés mais toujours aventureux.
Names: Jack, Jill
Output JSON: {"french_summary": "Jack et Jill partent à la recherche d'eau d'un puits au sommet d'une colline, mais ils tombent tous les deux et rentrent chez eux légèrement blessés mais toujours aventureux.", "num_names": 2}


### Tactic 2: Instruct the model to work out its own solution before rushing to a conclusion.

In [11]:
prompt = """
Determine if the student's solution is correct or not.

Question:
I'm building a solar power installation and I need \
 help working out the financials.
- Land costs $100 / square foot
- I can buy solar panels for $250 / square foot
- I negotiated a contract for maintenance that will cost \
me a flat $100k per year, and an additional $10 / square \
foot
What is the total cost for the first year of operations
as a function of the number of square feet.

Student's Solution:
Let x be the size of the installation in square feet.
Costs:
1. Land cost: 100x
2. Solar panel cost: 250x
3. Maintenance cost: 100,000 + 100x
Total cost: 100x + 250x + 100,000 + 100x = 450x + 100,000
"""

print(get_completion(prompt=prompt))

The student's solution is correct. They correctly identified the costs for land, solar panels, and maintenance, and added them together to find the total cost for the first year of operations as a function of the number of square feet.


Note that the student's solution is actually not correct.
We can fix this by instructing the model to work out its own solution first.

In [12]:
prompt = """
Your task is to determine if the student's solution \
is correct or not.
To solve the problem do the following:
- First, work out your own solution to the problem.
- Then compare your solution to the student's solution \
and evaluate if the student's solution is correct or not.
Don't decide if the student's solution is correct until
you have done the problem yourself.

Use the following format:
Question:
```
question here
```
Student's solution:
```
student's solution here
```
Actual solution:
```
steps to work out the solution and your solution here
```
Is the student's solution the same as actual solution \
just calculated:
```
yes or no
```
Student grade:
```
correct or incorrect
```

Question:
```
I'm building a solar power installation and I need help \
working out the financials.
- Land costs $100 / square foot
- I can buy solar panels for $250 / square foot
- I negotiated a contract for maintenance that will cost \
me a flat $100k per year, and an additional $10 / square \
foot
What is the total cost for the first year of operations \
as a function of the number of square feet.
```
Student's solution:
```
Let x be the size of the installation in square feet.
Costs:
1. Land cost: 100x
2. Solar panel cost: 250x
3. Maintenance cost: 100,000 + 100x
Total cost: 100x + 250x + 100,000 + 100x = 450x + 100,000
```
Actual solution:
"""

print(get_completion(prompt=prompt))

To calculate the total cost for the first year of operations, we need to add up the costs of land, solar panels, and maintenance.

1. Land cost: $100 / square foot
The cost of land is $100 multiplied by the number of square feet.

2. Solar panel cost: $250 / square foot
The cost of solar panels is $250 multiplied by the number of square feet.

3. Maintenance cost: $100,000 + $10 / square foot
The maintenance cost is a flat fee of $100,000 per year, plus $10 multiplied by the number of square feet.

Total cost: Land cost + Solar panel cost + Maintenance cost

So the actual solution is:
Total cost = (100 * x) + (250 * x) + (100,000 + (10 * x))

Is the student's solution the same as the actual solution just calculated:
Yes

Student grade:
Correct


## Model Limitations: Hallucinations

In [13]:
# Boie is a real company, the product name is not real.
prompt = """Tell me about AeroGlide UltraSlim Smart Toothbrush by Boie"""

print(get_completion(prompt=prompt))

The AeroGlide UltraSlim Smart Toothbrush is a revolutionary oral care product developed by Boie, a company known for its innovative approach to personal hygiene. This toothbrush is designed to provide a superior cleaning experience while being gentle on the gums and teeth.

One of the standout features of the AeroGlide UltraSlim Smart Toothbrush is its ultra-slim bristles. These bristles are only 0.04mm thick, making them incredibly soft and flexible. This design allows the toothbrush to reach even the most difficult-to-access areas of the mouth, ensuring a thorough and effective cleaning.

The toothbrush also incorporates smart technology to enhance the brushing experience. It has a built-in timer that vibrates every 30 seconds, reminding users to move to a different quadrant of their mouth. After two minutes, the toothbrush automatically shuts off, indicating the recommended brushing time has been reached.

Another notable feature of the AeroGlide UltraSlim Smart Toothbrush is its an